# Make Test Network

In [1]:
import os
import sys
import yaml
import glob
import copy

import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = os.path.join('C:\\Users', 'david.ory', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
boundary_dir = os.path.join(root_dir, 'data', 'external', 'census')
output_dir = input_dir
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = os.path.join('C:\\Users', 'david.ory', 'Documents', 'GitHub', 'mtc_Lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-01-15 14:19:13, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2021-01-15 14:19:13, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2021-01-15 14:19:13, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2021-01-15 14:19:13, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso


### Data Reads

In [7]:
link_file = os.path.join(input_dir, 'v03_link.json')
node_file = os.path.join(input_dir, 'v03_node.geojson')
shape_file = os.path.join(input_dir, 'v03_shape.geojson')

road_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

2021-01-15 14:19:28, INFO: Reading from following files:
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_link.json
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_node.geojson
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_shape.geojson.
2021-01-15 14:19:28, INFO: Reading from following files:
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_link.json
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_node.geojson
-C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_shape.geojson.
2021-01-15 14:26:36, INFO: Read 2238330 links from C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_03\v03_link.json
2021-01-15 14:26:36, INFO: Read 2238330 links from C:\Users\david.ory\Documents\GitHub\travel-model

ValueError: IDs in network not unique

In [ ]:
transit_net = TransitNetwork.read(feed_path = input_dir)

In [ ]:
places_gdf = gpd.read_file(os.path.join(boundary_dir, 'CA_Places_TIGER2016.shp'))
places_gdf.head()

In [ ]:
union_city_gdf = places_gdf.loc[places_gdf['NAME'] == 'Union City'].to_crs(4326)
union_city_gdf.head()

### Roadway Selection

In [ ]:
shapes_gdf = road_net.shapes_df

In [ ]:
shapes_gdf.head()

In [ ]:
union_city_shapes_gdf = gpd.sjoin(shapes_gdf, union_city_gdf, op='intersects')

In [ ]:
union_city_shapes_gdf.head()

In [ ]:
links_df = road_net.links_df
links_df.columns

In [ ]:
union_city_links_df = pd.merge(
    union_city_shapes_gdf[['id', 'shape_id']],
    links_df,
    on = 'shape_id',
    how = 'left',
)

In [ ]:
union_city_links_df.shape

In [ ]:
node_list_df = pd.concat(
    [union_city_links_df[['fromIntersectionId']].rename(columns = {'fromIntersectionId': 'shst_node_id'}), 
     union_city_links_df[['toIntersectionId']].rename(columns = {'toIntersectionId': 'shst_node_id'})], 
    ignore_index = True
).drop_duplicates()

node_list_df.head()

In [ ]:
nodes_df = road_net.nodes_df
nodes_df.head()

In [ ]:
nodes_df.shape

In [ ]:
union_city_nodes_df = nodes_df[nodes_df['shst_node_id'].isin(node_list_df['shst_node_id'])]
union_city_nodes_df.shape

### Transit Selections

In [ ]:
agency_df = transit_net.feed.agency.loc[transit_net.feed.agency['agency_id'] == 'UCT']
routes_df = transit_net.feed.routes.loc[transit_net.feed.routes['agency_id'] == 'UCT']
trips_df = transit_net.feed.trips[transit_net.feed.trips['route_id'].isin(routes_df['route_id'])]
freqs_df = transit_net.feed.frequencies[transit_net.feed.frequencies['trip_id'].isin(trips_df['trip_id'])]
shapes_df = transit_net.feed.shapes[transit_net.feed.shapes['shape_id'].isin(trips_df['shape_id'])]
stop_times_df = transit_net.feed.stop_times[transit_net.feed.stop_times['trip_id'].isin(trips_df['trip_id'])]
stops_df = transit_net.feed.stops[transit_net.feed.stops['stop_id'].isin(stop_times_df['stop_id'])]

### Make and Verify Network

In [ ]:
union_city_roadway_net = copy.deepcopy(road_net)
union_city_roadway_net.links_df = links_df
union_city_roadway_net.shapes_df = shapes_gdf
union_city_roadway_net.nodes_df = nodes_df

In [ ]:
union_city_transit_net =  copy.deepcopy(transit_net)
union_city_transit_net.feed.agency = agency_df
union_city_transit_net.feed.routes = routes_df
union_city_transit_net.feed.trips  = trips_df
union_city_transit_net.feed.frequencies = freqs_df
union_city_transit_net.feed.shapes = shapes_df
union_city_transit_net.feed.stop_times = stop_times_df
union_city_transit_net.feed.stops = stops_df

union_city_transit_net.road_net = union_city_roadway_net

In [ ]:
union_city_roadway_net.links_df.columns

In [ ]:
union_city_transit_net.validate_road_network_consistencies()

In [ ]:
union_city_transit_net.validate_transit_stops()

In [ ]:
# TODO: iteratively add nodes if needed to accommodate transit routes